In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('data/train/trainx.txt', sep='\t')
features = data.columns.values
print("特征如下：{} \n".format(features))
print("数据数目: ", len(data))


特征如下：['ITEM_NAME' 'TYPE'] 

数据数目:  499447


## 数据清洗
* 替换一些字符 类似于 （ ） ‘ ’ _

In [3]:
import re
from zhon.hanzi import punctuation
from string import punctuation as english_punc
def  rm_punc(strs):
    return re.sub(r"[{}]+".format(punctuation + english_punc)," ",strs)
data[features[0]]  = data[features[0]].apply(rm_punc)

data.head()

,ITEM_NAME,TYPE
0,腾讯QQ黄钻三个月QQ黄钻3个月季卡官方自动充值可查时间可续费,本地生活--游戏充值--QQ充值
1,腾讯q币充值30元qq币30qb30q币自动充值,本地生活--游戏充值--QQ充值
2,腾讯QQ红钻包月卡 QQ红钻一个月QQ红钻1个月,本地生活--游戏充值--QQ充值
3,腾讯QQ蓝钻贵族31个月直充,本地生活--游戏充值--QQ充值
4,腾讯QQ币148元148QQ币148个直充148Q币148个Q币148个QQB★自动充值,本地生活--游戏充值--QQ充值


In [4]:
# 因为有的label里本身就存在空格，所以先要把空格去掉，再去掉- - ， 再以空格划分
data = shuffle(data)
data_y = data[features[1]]
train = data.sample(frac= 0.4).reset_index()
test = data.sample(frac= 0.1).reset_index()
print("训练数据数目:  ", len(train))
print("测试数据数目:  ", len(test))



train_x  = train[features[0]]
train_y  = train[features[1]]

test_x  = test[features[0]]
test_y  = test[features[1]]

print(train_x.shape, test_x.shape)

data_y = data_y.apply(lambda x: x.replace(" ", "").replace("--", " ").replace("/", "")).tolist()
data_y= [each.split() for each in data_y]

train_y = train_y.apply(lambda x: x.replace(" ", "").replace("--", " ").replace("/", "")).tolist()
train_y = [each.split() for each in train_y]

test_y = test_y.apply(lambda x: x.replace(" ", "").replace("--", " ").replace("/", "")).tolist()
test_y = [each.split() for each in test_y]

训练数据数目:   199779
测试数据数目:   49945
(199779,) (49945,)


## 分词

In [5]:
import jieba
def participle(data):
    words = []
    for i in range(len(data)):
        result=[]
        seg_list = jieba.cut(data.iloc[i])
        for w in seg_list :#读取每一行分词
            if w != " ":
                result.append(w)
        words.append(result)#将该行分词写入列表形式的总分词列表
    return words

In [6]:
word_data = participle(data[features[0]])
word_data_train = participle(train_x)
word_data_test = participle(test_x)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Richado\AppData\Local\Temp\jieba.cache
Loading model cost 0.703 seconds.
Prefix dict has been built succesfully.


# 词向量训练

In [7]:
from gensim.models.word2vec import Word2Vec 
sentences = word_data
model= Word2Vec(size=50, window=10, min_count = 1)
model.build_vocab(sentences)
model.train(sentences,total_examples = model.corpus_count,epochs = model.iter)

c:\users\richado\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """


(43583239, 44588295)

In [8]:
model.save("w2v_model")

In [9]:
train_y = pd.DataFrame(train_y, columns=['label1', 'label2', 'label3'])
test_y = pd.DataFrame(test_y, columns=['label1', 'label2', 'label3'])
data_y = pd.DataFrame(data_y, columns=['label1', 'label2', 'label3'])

In [10]:
len(data_y["label1"].unique())

22

In [11]:
len(data_y["label2"].unique())

191

In [12]:
len(data_y["label3"].unique())

1192

In [13]:
from keras.utils import to_categorical
y_label1 = data_y['label1'].unique().tolist()
y_label2 = data_y['label2'].unique().tolist()
y_label3 = data_y['label3'].unique().tolist()

train_y_label1_map = train_y['label1'].apply(lambda x: y_label1.index(x))
train_y_label2_map = train_y['label2'].apply(lambda x: y_label2.index(x))
train_y_label3_map = train_y['label3'].apply(lambda x: y_label3.index(x))
test_y_label1_map = test_y['label1'].apply(lambda x: y_label1.index(x))
test_y_label2_map = test_y['label2'].apply(lambda x: y_label2.index(x))
test_y_label3_map = test_y['label3'].apply(lambda x: y_label3.index(x))

Using TensorFlow backend.


In [14]:
train_y_label1_one_hot = to_categorical(train_y_label1_map)
train_y_label2_one_hot = to_categorical(train_y_label2_map)
train_y_label3_one_hot = to_categorical(train_y_label3_map)
test_y_label1_one_hot = to_categorical(test_y_label1_map)
test_y_label2_one_hot = to_categorical(test_y_label2_map)
test_y_label3_one_hot = to_categorical(test_y_label3_map)

## TextCnn

In [15]:
from __future__ import print_function
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb
from keras import initializers
# set parameters:
# max_features = 1192
maxlen = 16
batch_size = 32
embedding_dims = 50
filters = 64
kernel_size = 5
hidden_dims = 128
epochs = 100


x_train = [model[word] for word in word_data_train]
x_test = [model[word] for word in word_data_test]
y_train = train_y_label2_one_hot
y_test = test_y_label2_one_hot

print('Pad sequences (samples x time)')

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test  = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', train_y_label1_one_hot.shape)
print('y_test shape:', test_y_label1_one_hot.shape)

c:\users\richado\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
c:\users\richado\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Pad sequences (samples x time)
x_train shape: (199779, 16, 50)
x_test shape: (49945, 16, 50)
y_train shape: (199779, 22)
y_test shape: (49945, 22)


In [19]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPool1D
from keras.datasets import imdb
from keras import initializers
from keras import Input
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.backend import *
from keras import Model

def test_cnn(y_shape,maxlen,embedding_dims,filters = 250):
    
    inputs = Input(shape=(maxlen,embedding_dims, ))
    convs = []
    filter_sizes = [2,3,4,5]
    
    for fsz in filter_sizes:
        conv1 = Conv1D(filters,kernel_size=fsz,activation='tanh',strides=1)(inputs)
        print(conv1.shape)
#         pool1 = MaxPooling1D(maxlen-fsz+1)(conv1)
#         print(pool1.shape)
#         pool1 = Flatten()(pool1)
        
        pool1 = MaxPool1D(pool_size=int(conv1.shape[1]))(conv1)
        print(pool1.shape)
        convs.append(pool1)
    merge = concatenate([p for p in convs])
    merge = Flatten()(merge)
    out = Dropout(0.5)(merge)
    output = Dense(32,activation='relu')(out)
    print(output.shape)
    output = Dense(y_shape,activation='softmax')(output)
    print(inputs.shape)
    print(output.shape)
    model = Model([inputs],[output])
    
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [16]:
import tensorflow as tf
import numpy as np


class TextCNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(
      self, maxlen, sequence_length, num_classes, filter_sizes, embedding_size, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.float32, [None, sequence_length, embedding_size], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
#         self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        # Embedding layer
#         with tf.device('/cpu:0'), tf.name_scope("embedding"):
#             self.W = tf.Variable(
#                 tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
#                 name="W")
#             self.embedded_chars = tf.nn.embedding_lookup(self.W, self.input_x)
#             self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

#         # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        self.input_x_expand = tf.expand_dims(self.input_x, -1)
        print(self.input_x_expand.shape)
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                print(W.shape)
                conv = tf.nn.conv2d(
                    self.input_x_expand,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(pooled_outputs, 3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])
        print(self.h_pool_flat.shape)
        # Add dropout
#         with tf.name_scope("dropout"):
#             self.h_drop = tf.nn.dropout(self.h_pool_flat, self.input_x)
#         print(self.h_pool_flat.shape, elf.h_drop.shape)
        self.h_drop = self.h_pool_flat
        # Final (unnormalized) scores and predictions
        
        with tf.name_scope("output"):
            W1 = tf.get_variable(
                "W1",
                shape=[num_filters_total, 64],
                initializer=tf.contrib.layers.xavier_initializer())
            b1 = tf.Variable(tf.constant(0.1, shape=[64]), name="b1")
            
            self.output = tf.nn.relu(tf.nn.xw_plus_b(self.h_drop, W1, b1, name="output"))
            
            W2 = tf.get_variable(
                "W2",
                shape=[64, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b2 = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b2")
            l2_loss += tf.nn.l2_loss(W1)
            l2_loss += tf.nn.l2_loss(b1)
            self.scores = tf.nn.xw_plus_b(self.output, W2, b2, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")
            print(self.predictions.shape, self.input_y.shape)
        # Calculate mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

In [17]:
max_features = 1192
maxlen = 16
batch_size = 32
embedding_dims = 50
filters = 64
kernel_size = 5
hidden_dims = 128
epochs = 100
# cnnmodel = TextCNN(maxlen = 16, sequence_length = 16, num_classes = 22, filter_sizes = [2, 3, 4], embedding_size = 50, num_filters = 64)

In [18]:
import math
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
  
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = np.min(((batch_num + 1) * batch_size, data_size))
            yield shuffled_data[start_index:end_index]
            

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
# import data_helpers
# from text_cnn import TextCNN
from tensorflow.contrib import learn

# Parameters
# ==================================================

# Data loading params
# tf.flags.DEFINE_float("dev_sample_percentage", .1, "Percentage of the training data to use for validation")
# tf.flags.DEFINE_string("positive_data_file", "./data/rt-polaritydata/rt-polarity.pos", "Data source for the positive data.")
# tf.flags.DEFINE_string("negative_data_file", "./data/rt-polaritydata/rt-polarity.neg", "Data source for the negative data.")

# Model Hyperparameters
# tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
# tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
# tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
# tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
# tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
# tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
# tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
# tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
# tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
# tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
# tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
# tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS

def train(x_train, y_train, x_dev, y_dev):
    # Training
    # ==================================================

    with tf.Graph().as_default():
        session_conf = tf.ConfigProto(
#           allow_soft_placement=FLAGS.allow_soft_placement,
#           log_device_placement=FLAGS.log_device_placement
        )
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            cnn = TextCNN(maxlen = 16, sequence_length = 16, num_classes = 191, filter_sizes = [1,2, 3, 4, 5], embedding_size = 50, num_filters = 64)

            # Define Training procedure
            global_step = tf.Variable(0, name="global_step", trainable=False)
            optimizer = tf.train.AdamOptimizer(1e-3)
            grads_and_vars = optimizer.compute_gradients(cnn.loss)
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

            # Initialize all variables
            sess.run(tf.global_variables_initializer())
            writer = tf.summary.FileWriter("fwwb",sess.graph)
            acc_sum = 0
            loss_sum = 0
            
            def train_step(x_batch, y_batch, loss_sum, acc_sum, interal = 10):
  
                """
                A single training step
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y: y_batch
                }
                _, step,  loss, accuracy = sess.run(
                    [train_op, global_step, cnn.loss, cnn.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                acc_sum += accuracy
                loss_sum += loss
                if step % interal == 0:
                    print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss_sum/interal, acc_sum/interal))
                    loss_sum = 0
                    acc_sum = 0
                return loss_sum, acc_sum

            def dev_step(x_batch, y_batch):
                """
                Evaluates model on a dev set
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y: y_batch
                }
                step,  loss, accuracy = sess.run(
                    [global_step,cnn.loss, cnn.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
#                 return time_str, step, loss, accuracy
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))

            # Generate batches
            batches = batch_iter(
                list(zip(x_train, y_train)), 64, 200)
            # Training loop. For each batch...
            for batch in batches:
                x_batch, y_batch = zip(*batch)
                loss_sum, acc_sum = train_step(x_batch, y_batch,  loss_sum, acc_sum, 40)
                current_step = tf.train.global_step(sess, global_step)
                if current_step % 200000 == 0:
                    print("\nEvaluation:")
                    dev_step(x_dev, y_dev)
                    print("")
#                 if current_step % FLAGS.checkpoint_every == 0:
#                     path = saver.save(sess, checkpoint_prefix, global_step=current_step)
#                     print("Saved model checkpoint to {}\n".format(path))
            


train(x_train, y_train,  x_test, y_test)

(?, 16, 50, 1)
(1, 50, 1, 64)
(2, 50, 1, 64)
(3, 50, 1, 64)
(4, 50, 1, 64)
(5, 50, 1, 64)
(?, 320)
(?,) (?, 191)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

2019-02-18T19:43:31.286776: step 40, loss 3.6746, acc 0.314063
2019-02-18T19:43:31.612902: step 80, loss 2.37026, acc 0.516797
2019-02-18T19:43:31.935040: step 120, loss 2.03098, acc 0.585547
2019-02-18T19:43:32.265158: step 160, loss 1.79953, acc 0.619141
2019-02-18T19:43:32.604249: step 200, loss 1.68959, acc 0.636719
2019-02-18T19:43:32.919407: step 240, loss 1.54518, acc 0.663281
2019-02-18T19:43:33.202678: step 280, loss 1.4939, acc 0.665234
2019-02-18T19:43:33.499310: step 320, loss 1.36521, acc 0.691406
2019-02-18T19:43:33.784664: step 360, loss 1.31487, acc 0.708594
2019-02-18T19:43:34.066776: step 400, loss 1.29397, acc 0.709766
2019-02-18T19:43:34.348661: step 440, loss 1.25547

2019-02-18T19:44:05.583577: step 4720, loss 0.784726, acc 0.794531
2019-02-18T19:44:05.864653: step 4760, loss 0.893919, acc 0.776953
2019-02-18T19:44:06.152995: step 4800, loss 0.875753, acc 0.771484
2019-02-18T19:44:06.435140: step 4840, loss 0.835511, acc 0.788281
2019-02-18T19:44:06.721359: step 4880, loss 0.793784, acc 0.798828
2019-02-18T19:44:07.021579: step 4920, loss 0.777497, acc 0.801953
2019-02-18T19:44:07.315678: step 4960, loss 0.75645, acc 0.807031
2019-02-18T19:44:07.612647: step 5000, loss 0.847734, acc 0.779687
2019-02-18T19:44:07.900037: step 5040, loss 0.796347, acc 0.7875
2019-02-18T19:44:08.181496: step 5080, loss 0.788555, acc 0.796484
2019-02-18T19:44:08.462682: step 5120, loss 0.832164, acc 0.785156
2019-02-18T19:44:08.757602: step 5160, loss 0.813537, acc 0.789844
2019-02-18T19:44:09.047549: step 5200, loss 0.791716, acc 0.799219
2019-02-18T19:44:09.331616: step 5240, loss 0.817287, acc 0.792578
2019-02-18T19:44:09.614401: step 5280, loss 0.832101, acc 0.79296

2019-02-18T19:44:41.480634: step 9640, loss 0.682305, acc 0.814063
2019-02-18T19:44:41.783823: step 9680, loss 0.681716, acc 0.816797
2019-02-18T19:44:42.093750: step 9720, loss 0.693346, acc 0.824609
2019-02-18T19:44:42.396893: step 9760, loss 0.662592, acc 0.832031
2019-02-18T19:44:42.689129: step 9800, loss 0.655202, acc 0.823047
2019-02-18T19:44:42.973376: step 9840, loss 0.736627, acc 0.813281
2019-02-18T19:44:43.281564: step 9880, loss 0.728474, acc 0.812109
2019-02-18T19:44:43.587707: step 9920, loss 0.70162, acc 0.813281
2019-02-18T19:44:43.895883: step 9960, loss 0.704799, acc 0.814063
2019-02-18T19:44:44.195082: step 10000, loss 0.707301, acc 0.817969
2019-02-18T19:44:44.497315: step 10040, loss 0.718967, acc 0.811719
2019-02-18T19:44:44.793480: step 10080, loss 0.729148, acc 0.796875
2019-02-18T19:44:45.080779: step 10120, loss 0.657823, acc 0.817969
2019-02-18T19:44:45.368940: step 10160, loss 0.704849, acc 0.821484
2019-02-18T19:44:45.671181: step 10200, loss 0.664974, acc

2019-02-18T19:45:18.709970: step 14520, loss 0.696898, acc 0.808594
2019-02-18T19:45:19.049060: step 14560, loss 0.661499, acc 0.814453
2019-02-18T19:45:19.372198: step 14600, loss 0.674288, acc 0.814844
2019-02-18T19:45:19.683364: step 14640, loss 0.675013, acc 0.817969
2019-02-18T19:45:19.983560: step 14680, loss 0.701684, acc 0.813672
2019-02-18T19:45:20.295724: step 14720, loss 0.695099, acc 0.822656
2019-02-18T19:45:20.611929: step 14760, loss 0.634083, acc 0.824609
2019-02-18T19:45:20.926086: step 14800, loss 0.650078, acc 0.825781
2019-02-18T19:45:21.237247: step 14840, loss 0.668458, acc 0.823047
2019-02-18T19:45:21.548374: step 14880, loss 0.710603, acc 0.817187
2019-02-18T19:45:21.847575: step 14920, loss 0.707325, acc 0.812109
2019-02-18T19:45:22.148767: step 14960, loss 0.718225, acc 0.814844
2019-02-18T19:45:22.463972: step 15000, loss 0.690772, acc 0.823828
2019-02-18T19:45:22.765117: step 15040, loss 0.723092, acc 0.811719
2019-02-18T19:45:23.068310: step 15080, loss 0.7

2019-02-18T19:45:54.719484: step 19360, loss 0.566918, acc 0.844141
2019-02-18T19:45:55.000679: step 19400, loss 0.576967, acc 0.839063
2019-02-18T19:45:55.281862: step 19440, loss 0.613349, acc 0.825
2019-02-18T19:45:55.563067: step 19480, loss 0.613513, acc 0.8375
2019-02-18T19:45:55.829375: step 19520, loss 0.670873, acc 0.820703
2019-02-18T19:45:56.119818: step 19560, loss 0.615548, acc 0.832031
2019-02-18T19:45:56.401002: step 19600, loss 0.593539, acc 0.849609
2019-02-18T19:45:56.671971: step 19640, loss 0.658158, acc 0.832422
2019-02-18T19:45:56.953117: step 19680, loss 0.597532, acc 0.830859
2019-02-18T19:45:57.234339: step 19720, loss 0.600703, acc 0.8375
2019-02-18T19:45:57.515519: step 19760, loss 0.681495, acc 0.819922
2019-02-18T19:45:57.798146: step 19800, loss 0.630453, acc 0.833594
2019-02-18T19:45:58.094988: step 19840, loss 0.633493, acc 0.829297
2019-02-18T19:45:58.376188: step 19880, loss 0.581811, acc 0.841406
2019-02-18T19:45:58.659119: step 19920, loss 0.673087, 

...

2019-02-18T20:29:14.132994: step 351240, loss 0.369035, acc 0.891016
2019-02-18T20:29:14.517577: step 351280, loss 0.364611, acc 0.894922
2019-02-18T20:29:14.902579: step 351320, loss 0.383435, acc 0.885156
2019-02-18T20:29:15.286598: step 351360, loss 0.370739, acc 0.885547
2019-02-18T20:29:15.671450: step 351400, loss 0.404641, acc 0.885938
2019-02-18T20:29:16.054789: step 351440, loss 0.392378, acc 0.882031
2019-02-18T20:29:16.438194: step 351480, loss 0.374456, acc 0.889844
2019-02-18T20:29:16.824618: step 351520, loss 0.433601, acc 0.878516
2019-02-18T20:29:17.209642: step 351560, loss 0.363942, acc 0.889844
2019-02-18T20:29:17.594900: step 351600, loss 0.395758, acc 0.886719
2019-02-18T20:29:17.978853: step 351640, loss 0.379879, acc 0.889062
2019-02-18T20:29:18.363408: step 351680, loss 0.368508, acc 0.891406
2019-02-18T20:29:18.749377: step 351720, loss 0.39048, acc 0.885156
2019-02-18T20:29:19.133142: step 351760, loss 0.381837, acc 0.88125
2019-02-18T20:29:19.517859: step 351

2019-02-18T20:30:01.227086: step 356040, loss 0.317015, acc 0.894141
2019-02-18T20:30:01.612716: step 356080, loss 0.294426, acc 0.905859
2019-02-18T20:30:01.999046: step 356120, loss 0.313399, acc 0.90625
2019-02-18T20:30:02.383871: step 356160, loss 0.312656, acc 0.900781
2019-02-18T20:30:02.769681: step 356200, loss 0.293474, acc 0.905469
2019-02-18T20:30:03.152924: step 356240, loss 0.365023, acc 0.884766
2019-02-18T20:30:03.538257: step 356280, loss 0.3268, acc 0.900391
2019-02-18T20:30:03.925656: step 356320, loss 0.329267, acc 0.903906
2019-02-18T20:30:04.310465: step 356360, loss 0.319801, acc 0.903906
2019-02-18T20:30:04.695606: step 356400, loss 0.315193, acc 0.906641
2019-02-18T20:30:05.078201: step 356440, loss 0.302531, acc 0.907422
2019-02-18T20:30:05.462742: step 356480, loss 0.308737, acc 0.907813
2019-02-18T20:30:05.849468: step 356520, loss 0.349411, acc 0.896094
2019-02-18T20:30:06.234135: step 356560, loss 0.35772, acc 0.896875
2019-02-18T20:30:06.618716: step 35660

2019-02-18T20:30:47.193227: step 360800, loss 0.382923, acc 0.885938
2019-02-18T20:30:47.576370: step 360840, loss 0.385661, acc 0.885938
2019-02-18T20:30:47.959852: step 360880, loss 0.351802, acc 0.889844
2019-02-18T20:30:48.345371: step 360920, loss 0.387659, acc 0.883594
2019-02-18T20:30:48.729258: step 360960, loss 0.422851, acc 0.878125
2019-02-18T20:30:49.113506: step 361000, loss 0.406477, acc 0.88125
2019-02-18T20:30:49.496797: step 361040, loss 0.347763, acc 0.89375
2019-02-18T20:30:49.881603: step 361080, loss 0.374132, acc 0.888672
2019-02-18T20:30:50.268470: step 361120, loss 0.432964, acc 0.873828
2019-02-18T20:30:50.654442: step 361160, loss 0.401101, acc 0.876563
2019-02-18T20:30:51.036652: step 361200, loss 0.360143, acc 0.8875
2019-02-18T20:30:51.422790: step 361240, loss 0.416058, acc 0.875
2019-02-18T20:30:51.807460: step 361280, loss 0.425614, acc 0.876953
2019-02-18T20:30:52.192201: step 361320, loss 0.400721, acc 0.886328
2019-02-18T20:30:52.577628: step 361360, 

2019-02-18T20:31:35.526768: step 365600, loss 0.330347, acc 0.900391
2019-02-18T20:31:35.911811: step 365640, loss 0.344862, acc 0.895703
2019-02-18T20:31:36.296551: step 365680, loss 0.325482, acc 0.899219
2019-02-18T20:31:36.683374: step 365720, loss 0.323602, acc 0.907422
2019-02-18T20:31:37.072717: step 365760, loss 0.322014, acc 0.899609
2019-02-18T20:31:37.458240: step 365800, loss 0.332643, acc 0.9
2019-02-18T20:31:37.843507: step 365840, loss 0.360142, acc 0.902344
2019-02-18T20:31:38.230321: step 365880, loss 0.362171, acc 0.894141
2019-02-18T20:31:38.614694: step 365920, loss 0.376678, acc 0.886328
2019-02-18T20:31:39.000372: step 365960, loss 0.322623, acc 0.904687
2019-02-18T20:31:39.386219: step 366000, loss 0.342206, acc 0.897656
2019-02-18T20:31:39.770107: step 366040, loss 0.372118, acc 0.896094
2019-02-18T20:31:40.156278: step 366080, loss 0.353214, acc 0.893359
2019-02-18T20:31:40.541056: step 366120, loss 0.342733, acc 0.9
2019-02-18T20:31:40.924556: step 366160, los

2019-02-18T20:32:26.110025: step 370400, loss 0.41083, acc 0.88125
2019-02-18T20:32:26.575638: step 370440, loss 0.398264, acc 0.887891
2019-02-18T20:32:27.046824: step 370480, loss 0.405267, acc 0.871484
2019-02-18T20:32:27.513929: step 370520, loss 0.398074, acc 0.882422
2019-02-18T20:32:27.981001: step 370560, loss 0.365246, acc 0.887109
2019-02-18T20:32:28.447906: step 370600, loss 0.401413, acc 0.879297
2019-02-18T20:32:28.916651: step 370640, loss 0.403063, acc 0.883594
2019-02-18T20:32:29.383156: step 370680, loss 0.400015, acc 0.880859
2019-02-18T20:32:29.848608: step 370720, loss 0.433505, acc 0.869141
2019-02-18T20:32:30.313661: step 370760, loss 0.39503, acc 0.878516
2019-02-18T20:32:30.779452: step 370800, loss 0.409063, acc 0.875781
2019-02-18T20:32:31.247466: step 370840, loss 0.372991, acc 0.886719
2019-02-18T20:32:31.713941: step 370880, loss 0.440731, acc 0.875
2019-02-18T20:32:32.183192: step 370920, loss 0.408692, acc 0.880469
2019-02-18T20:32:32.650251: step 370960,